# Naive bayes


In [ ]:
import re
import glob 
import math

from collections import defaultdict
from pprint import pprint
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

## Dataset
Here we have downloaded a dataset from SpamAssassin

In [66]:
path = './dataset/*/*/*'  # wildcard, glob argument, indicates path for files
data = []

In [67]:
for fn in glob.glob(path)[:5]: 
    with open(fn, 'r') as file:
        pprint(file.read())


('Received: from mail.ilxresorts.com (mail.ilxresorts.com [207.108.153.250])\n'
 '\tby linux.midrange.com (8.11.6/8.11.6) with ESMTP id g6MJU8e16448\n'
 '\tfor <gibbs@midrange.com>; Mon, 22 Jul 2002 14:30:09 -0500\n'
 'Received: from ilx2kweb (www.ilxresorts.com [207.108.153.247])\n'
 '\tby mail.ilxresorts.com (8.9.3/8.9.3) with SMTP id NAA01973\n'
 '\tfor <gibbs@midrange.com>; Mon, 22 Jul 2002 13:20:35 -0700\n'
 'From: blowdamovie@atlas.cz\n'
 'Message-Id: <200207222020.NAA01973@mail.ilxresorts.com>\n'
 'X-Mailer: DevMailer v1.0 (http://www.geocel.com/devmailer/)\n'
 'Date: Mon, 22 Jul 2002 12:37:15 US Mountain Standard Time\n'
 'To: gibbs@midrange.com\n'
 'Subject: Life-Time upgrades for FREEq4i1i6p8\n'
 'MIME-Version: 1.0\n'
 'Content-type: multipart/mixed; boundary="#DM659823986#"\n'
 'X-Status: \n'
 'X-Keywords: \n'
 '\n'
 '\n'
 '--#DM659823986#\n'
 'Content-Type: text/plain; charset=us-ascii\n'
 'Content-Transfer-Encoding: quoted-printable\n'
 '\n'
 'Below is the result of your f

## Preprocessing
We just get the header of the email, and if it's spam or not

In [68]:
for fn in glob.glob(path):
    is_spam = 'ham' not in fn  # the relative path, the dir it's named ham if it's not spam
    with open(fn, 'r') as file:
        try:
            for line in file:
                if line.startswith('Subject: '):  # get just the subject
                    subject = re.sub(f'^Subject: ', '', line).strip()  # remove the leading subject
                    data.append((subject, is_spam))
        except UnicodeDecodeError:
            print(fn, 'UnicodeDecodeError (skip)')

./dataset/spam_2/spam_2/00645.dd7d8ec1eb687c5966c516b720fcc3d5 UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/00886.9bd2063c3d984a66958a6195ffb97849 UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/01013.c6cf4f54eda63230389baccc02702034 UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/00043.9331daf0bd865aa657cb02cbcd06173b UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/00271.7105f4998a88cbf4036403f61ba60d65 UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/00136.870132877ae18f6129c09da3a4d077af UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/01233.010677fced50f4aecb67fba70701bcf5 UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/01072.ac604802c74de2ebc445efc827299b96 UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/01379.0d39498608cd170bbbc8cd33ffd18e35 UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/00951.f7044a1b178dc3dcff44932f840b8805 UnicodeDecodeError (skip)
./dataset/spam_2/spam_2/00214.39bd955c9db013255c326dbcbb4f2f86 UnicodeDecodeError (skip)
./dataset/spam_2/spam

In [69]:
pprint(data[:5])

[('Life-Time upgrades for FREEq4i1i6p8', True),
 ('Protect Your Computer With Norton Systemworks 2002', True),
 ('We BUY Your NEW & USED Magnetic Media or Backup Data Cartridges', True),
 ('Build a great future', True),
 ('You Won $30 Dollars !', True)]


## Math background

### Total probability theorem
Let $A$ be a partition, for any event $B$ we have
$$ \bigcap_i A_i = \emptyset $$
$$ B = \bigcup_i (A_i \cap B) $$
Therefore
$$ P(B) = \sum P(B|A_i)P(A_i) $$

### Bayes theorem
$$ P(A|B) = \frac{P(A \cap B)}{P(B)} = \frac{P(B|A)P(A)}{P(B)} $$

### Naive bayes
For this approach we are interested in knowing the probability of some event $S$ given some features $x_1,\dots,x_n$, mathematically $P(S|X_1=x_1,\dots,X_n=x_n)$. Using the bayes theorem we have

$$ P(S|X) = \frac{P(X|S)P(S)}{P(X)} = \frac{P(X|S)P(S)}{P(X|S)P(S) + P(X|\overline S)P(\overline S)} $$

In naive bayes, we make the assumption that $x_i$ are independent each other and $P(S)=P(\overline S)$. With this assumptions we have that

$$P(X_1=x_1,\dots,X_n=x_n|S) = \prod P(X_i=x_i|S)$$
$$P(S|X) = \frac{P(X|S)}{P(X|S) + P(X|\overline S)} $$

To compute $P(X_i=x_i|S)$ we use

$$P(X_i=x_i|S) = \frac{k + |\{x | x \in S\}|}{2k + |S|} $$

We use the pseudocount $k$ to smoth the probability.

### Float-point calculation
$$ \prod p_i = \exp(\sum log (p_i)) $$

In [70]:
def tokenize(msg):
    msg = msg.lower()
    words = re.findall("[a-z0-9']+", msg)  
    return set(words)  # remove duplicates

In [71]:
print(data[0][0])
tokenize(data[0][0])

Life-Time upgrades for FREEq4i1i6p8


{'for', 'freeq4i1i6p8', 'life', 'time', 'upgrades'}

In [72]:
def count_words(X_train, y_train):
    counts = defaultdict(lambda: [0, 0])  # by default a list [0, 0]
    for text, is_spam in zip(X_train, y_train):
        for word in tokenize(text):
            counts[word][0 if is_spam else 1] += 1
    return counts

In [78]:
def word_probs(counts, spam_count, non_spam_count, k):
    return [(w, 
             (w_in_spam + k) / (2*k + spam_count), 
             (w_in_non_spam + k) / (2*k + non_spam_count))
             for w, (w_in_spam, w_in_non_spam) in counts.items()]

In [74]:
def word_proba(word_probs, msg, k=0.5):
    predictable_words = tokenize(msg)
    log_prob_if_spam = 0.0
    log_prob_if_not_spam = 0.0
    for w, p_spam, p_not_spam in word_probs:
        if w in predictable_words:
            log_prob_if_spam += math.log(p_spam)
            log_prob_if_not_spam += math.log(p_not_spam)
        else:
            log_prob_if_spam += math.log(1 - p_spam)
            log_prob_if_not_spam += math.log(1 - p_not_spam)
    prob_spam = math.exp(log_prob_if_spam)
    prob_not_spam = math.exp(log_prob_if_not_spam)
    return prob_spam / (prob_spam + prob_not_spam)

In [75]:
class BernoulliNB:
    
    def fit(self, X_train, y_train, k=0.5):
        counts = count_words(X_train, y_train)
        spam_count = len([y for y in y_train if y])
        not_spam_count = len(y_train) - spam_count
        self.word_probs = word_probs(counts, spam_count, not_spam_count, k)
        self.k = k
        
    def transform(self, X_test):
        return [word_proba(self.word_probs, msg, self.k) for msg in X_test]

In [76]:
X = [d[0] for d in data]
y = [d[1] for d in data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [81]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.transform(X_test)

In [83]:
y_pred = [True if y > 0.5 else False for y in y_pred]

In [94]:
print(confusion_matrix(y_test, y_pred))
f1_score(y_test, y_pred)

[[744  62]
 [ 68 388]]


0.8565121412803532